# 📗 발표용 Part 3: 모델링 및 최적화

**프로젝트**: 한국 기업 부도 예측 시스템  
**작성일**: 2025년  
**목표**: 도메인 기반 특성을 활용한 불균형 분류 모델링 및 비즈니스 적용

---

## 🎯 Part 3 목표 및 이전 Part 요약

### Part 1-2 주요 발견

**Part 1: 문제 정의 및 탐색적 분석**
- 유동성이 가장 강력한 예측 변수 (유동비율, 당좌비율, 현금비율)
- 업종별 부도율 2배 차이 (건설업 2.8% vs 금융업 0.9%)
- 외감 여부가 부도율에 영향

**Part 2: 도메인 특성 공학**
- **생성된 특성**: 52개 → **최종 선택**: 27개 (다중공선성 제거 + 예측력 검증)
- **Feature Validation**: Mann-Whitney U test, Cliff's Delta, AUC 기반 검증
- **VIF 다중공선성 분석**: 고VIF 특성 14개 제거
- **IV 기반 선택**: Information Value > 0.02 기준
- **출력 파일**: `domain_based_features_완전판.csv` (50,105 rows × 27 features)

### Part 3 목표

1. ✅ **불균형 데이터 처리**: 5가지 리샘플링 전략 (SMOTE, BorderlineSMOTE, RandomUnderSampler, SMOTETomek)
2. ✅ **AutoML 최적화**: RandomizedSearchCV로 5개 모델 × 100회 랜덤 샘플링
3. ✅ **앙상블 모델**: Weighted Voting (Top 3 모델 기반)
4. ✅ **비즈니스 적용**: Traffic Light 시스템 (동적 임계값 기반 3등급 분류)
5. ✅ **모델 저장**: Part 4 SHAP 분석을 위한 파일 출력

---

## 0. 환경 설정

### 라이브러리 Import 및 한글 폰트 설정

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import os
import joblib
import platform
from datetime import datetime

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (OS별 자동 선택)
if platform.system() == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    plt.rc('font', family='NanumGothic')

plt.rc('axes', unicode_minus=False)  # 마이너스 기호 깨짐 방지

print("✅ 한글 폰트 설정 완료:", plt.rcParams['font.family'])

✅ 한글 폰트 설정 완료: ['AppleGothic']


In [2]:
# Scikit-learn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

# Imbalanced-learn
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier

# Gradient Boosting 모델
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# 평가 메트릭
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, precision_recall_curve, roc_curve
)

print("✅ 모든 라이브러리 import 완료")
print(f"Scikit-learn version: {__import__('sklearn').__version__}")
print(f"Imbalanced-learn version: {__import__('imblearn').__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"XGBoost version: {xgb.__version__}")
print(f"CatBoost version: {__import__('catboost').__version__}")

✅ 모든 라이브러리 import 완료
Scikit-learn version: 1.7.2
Imbalanced-learn version: 0.14.0
LightGBM version: 4.6.0
XGBoost version: 3.1.1
CatBoost version: 1.2.8


## 1. 데이터 로딩 및 분할

Part 2에서 생성한 도메인 기반 특성을 로딩합니다.

In [3]:
# Part 2 출력 파일 경로
DATA_PATH = '../data/features/domain_based_features_완전판.csv'
SAVE_DIR = '../data/processed/'

# 디렉토리 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# 데이터 로딩
print("📂 데이터 로딩 중...")
df = pd.read_csv(DATA_PATH, encoding='utf-8')

print(f"\n✅ 데이터 로딩 완료")
print(f"   - Shape: {df.shape}")
print(f"   - 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

📂 데이터 로딩 중...

✅ 데이터 로딩 완료
   - Shape: (50105, 28)
   - 메모리 사용량: 10.70 MB


In [4]:
# 타겟 변수명 설정
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'

# 데이터 정보 확인
print("📊 데이터 기본 정보")
print("=" * 60)
print(f"전체 기업 수: {len(df):,}")
print(f"특성 개수: {df.shape[1] - 1}")
print(f"\n부도율:")
print(df[TARGET_COL].value_counts())
print(f"\n부도율 비율:")
print(df[TARGET_COL].value_counts(normalize=True) * 100)

# 불균형 비율
imbalance_ratio = (df[TARGET_COL] == 0).sum() / (df[TARGET_COL] == 1).sum()
print(f"\n⚠️  불균형 비율: 1:{imbalance_ratio:.1f} (정상:부도)")

# 결측치 확인
missing_count = df.isnull().sum().sum()
print(f"\n결측치 총 개수: {missing_count:,}")
if missing_count > 0:
    print(f"결측치가 있는 컬럼 수: {(df.isnull().sum() > 0).sum()}")

📊 데이터 기본 정보
전체 기업 수: 50,105
특성 개수: 27

부도율:
모형개발용Performance(향후1년내부도여부)
0    49346
1      759
Name: count, dtype: int64

부도율 비율:
모형개발용Performance(향후1년내부도여부)
0    98.485181
1     1.514819
Name: proportion, dtype: float64

⚠️  불균형 비율: 1:65.0 (정상:부도)

결측치 총 개수: 0


In [5]:
# Train/Test Split (80:20)
RANDOM_STATE = 42
TEST_SIZE = 0.2

# X, y 분리
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Stratified split (부도율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    stratify=y, 
    random_state=RANDOM_STATE
)

print("\n✅ Train/Test Split 완료")
print("=" * 60)
print(f"Train Set: {X_train.shape} (부도율: {y_train.mean()*100:.2f}%)")
print(f"Test Set:  {X_test.shape} (부도율: {y_test.mean()*100:.2f}%)")
print(f"\n특성 목록 (총 {X_train.shape[1]}개):")
for i, col in enumerate(X_train.columns, 1):
    print(f"  {i}. {col}")


✅ Train/Test Split 완료
Train Set: (40084, 27) (부도율: 1.51%)
Test Set:  (10021, 27) (부도율: 1.52%)

특성 목록 (총 27개):
  1. 순부채비율
  2. 운전자본
  3. 이해관계자_불신지수
  4. 운전자본비율
  5. 이자부담률
  6. 공공정보리스크
  7. 판관비효율성
  8. 재고회전율
  9. 유동성압박지수
  10. 매출총이익률
  11. OCF_대_유동부채
  12. 부채레버리지
  13. 재고보유일수
  14. 현금소진일수
  15. 매출집중도
  16. 연체심각도
  17. 신용등급점수
  18. 부채상환년수
  19. 매출채권_이상지표
  20. 매출채권회전율
  21. 총발생액
  22. 현금흐름품질
  23. 긴급유동성
  24. 즉각지급능력
  25. 운전자본_대_자산
  26. 이자보상배율
  27. 현금창출능력


## 2. 전처리 클래스 정의

ImbPipeline에서 사용할 커스텀 전처리 클래스를 정의합니다.

In [6]:
class InfiniteHandler(BaseEstimator, TransformerMixin):
    """무한대 값(inf, -inf)을 NaN으로 변환"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy = X_copy.replace([np.inf, -np.inf], np.nan)
        return X_copy


class Winsorizer(BaseEstimator, TransformerMixin):
    """이상치를 특정 백분위수로 클리핑 (Winsorizing)"""
    
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.lower_bounds_ = None
        self.upper_bounds_ = None
    
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        self.lower_bounds_ = X_df.quantile(self.lower_quantile)
        self.upper_bounds_ = X_df.quantile(self.upper_quantile)
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        X_clipped = X_df.clip(lower=self.lower_bounds_, upper=self.upper_bounds_, axis=1)
        return X_clipped.values if not isinstance(X, pd.DataFrame) else X_clipped


class LogTransformer(BaseEstimator, TransformerMixin):
    """양수 값에만 로그 변환 적용 (log1p)"""
    
    def __init__(self, threshold=0):
        self.threshold = threshold
        self.positive_cols_ = None
    
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        # 양수만 있는 컬럼 찾기
        self.positive_cols_ = [col for col in X_df.columns if (X_df[col] > self.threshold).all()]
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        for col in self.positive_cols_:
            if col in X_df.columns:
                X_df[col] = np.log1p(X_df[col])
        return X_df.values if not isinstance(X, pd.DataFrame) else X_df


print("✅ 전처리 클래스 정의 완료")
print("   - InfiniteHandler: 무한대 값 → NaN")
print("   - Winsorizer: 이상치 클리핑 (1%~99% 백분위수)")
print("   - LogTransformer: 양수 컬럼에 log1p 변환")

✅ 전처리 클래스 정의 완료
   - InfiniteHandler: 무한대 값 → NaN
   - Winsorizer: 이상치 클리핑 (1%~99% 백분위수)
   - LogTransformer: 양수 컬럼에 log1p 변환


## 3. 불균형 데이터 처리 전략

### ImbPipeline 6단계 구조

```
1. InfiniteHandler    → 무한대 값 처리
2. Winsorizer         → 이상치 제어 (1%~99% 분위수)
3. LogTransformer     → 로그 변환 (양수만)
4. IterativeImputer   → 결측치 보간
5. RobustScaler       → 스케일링 (중앙값 기반)
6. Resampler          → 리샘플링 (5가지 전략)
7. Classifier         → 분류기 (5개 모델)
```

### 5가지 리샘플링 전략

| 전략 | 설명 | Sampling Ratio |
|------|------|----------------|
| **passthrough** | 리샘플링 없음 (베이스라인) | - |
| **SMOTE** | 소수 클래스 합성 샘플 생성 | 0.2 (20%) |
| **BorderlineSMOTE** | 경계선 샘플 중심 생성 | 0.2 (20%) |
| **RandomUnderSampler** | 다수 클래스 무작위 제거 | 0.3 (30%) |
| **SMOTETomek** | SMOTE + Tomek Links 정리 | 0.2 (20%) |

**Sampling Ratio 근거**:
- 원본 부도율: 1.51% (1:66)
- 목표 부도율: 20% (1:5) → 과생성 방지하면서도 학습 효과 보장
- 언더샘플링: 30%까지만 → 정상 기업 데이터 손실 최소화

In [7]:
# 5가지 리샘플링 전략 정의
SAMPLING_RATIO_OVER = 0.2   # SMOTE 계열: 20%
SAMPLING_RATIO_UNDER = 0.3  # 언더샘플링: 30%

resampler_list = [
    'passthrough',  # 1. 리샘플링 없음
    SMOTE(sampling_strategy=SAMPLING_RATIO_OVER, random_state=RANDOM_STATE),  # 2. SMOTE
    BorderlineSMOTE(sampling_strategy=SAMPLING_RATIO_OVER, random_state=RANDOM_STATE),  # 3. BorderlineSMOTE
    RandomUnderSampler(sampling_strategy=SAMPLING_RATIO_UNDER, random_state=RANDOM_STATE),  # 4. 언더샘플링
    SMOTETomek(sampling_strategy=SAMPLING_RATIO_OVER, random_state=RANDOM_STATE)  # 5. SMOTETomek
]

print("✅ 리샘플링 전략 정의 완료")
print("=" * 60)
for i, resampler in enumerate(resampler_list, 1):
    if resampler == 'passthrough':
        print(f"{i}. passthrough (리샘플링 없음)")
    else:
        print(f"{i}. {type(resampler).__name__} (ratio={resampler.sampling_strategy})")

✅ 리샘플링 전략 정의 완료
1. passthrough (리샘플링 없음)
2. SMOTE (ratio=0.2)
3. BorderlineSMOTE (ratio=0.2)
4. RandomUnderSampler (ratio=0.3)
5. SMOTETomek (ratio=0.2)


In [8]:
# 베이스 파이프라인 정의 (리샘플링 + 분류기는 AutoML에서 선택)
def create_pipeline():
    """ImbPipeline 생성 (전처리 6단계 + 리샘플링 + 분류기)"""
    pipeline = ImbPipeline([
        ('inf_handler', InfiniteHandler()),
        ('winsorizer', Winsorizer(lower_quantile=0.01, upper_quantile=0.99)),
        ('log_transformer', LogTransformer()),
        ('imputer', IterativeImputer(max_iter=10, random_state=RANDOM_STATE)),
        ('scaler', RobustScaler()),
        ('resampler', 'passthrough'),  # AutoML에서 선택
        ('classifier', LogisticRegression(random_state=RANDOM_STATE))  # AutoML에서 선택
    ])
    return pipeline

# 파이프라인 테스트
test_pipeline = create_pipeline()
print("\n✅ ImbPipeline 생성 함수 정의 완료")
print(f"\n파이프라인 단계:")
for step_name, step_obj in test_pipeline.steps:
    print(f"  - {step_name}: {type(step_obj).__name__}")


✅ ImbPipeline 생성 함수 정의 완료

파이프라인 단계:
  - inf_handler: InfiniteHandler
  - winsorizer: Winsorizer
  - log_transformer: LogTransformer
  - imputer: IterativeImputer
  - scaler: RobustScaler
  - resampler: str
  - classifier: LogisticRegression


## 4. AutoML: 하이퍼파라미터 튜닝

### RandomizedSearchCV 설정

- **탐색 공간**: 5개 모델 × 5개 리샘플링 전략 × 다양한 하이퍼파라미터
- **샘플링 횟수**: 100회 랜덤 샘플링
- **교차 검증**: StratifiedKFold 5-Fold
- **평가 메트릭**: PR-AUC (average_precision)
- **병렬 처리**: n_jobs=-1 (모든 CPU 코어 사용)

### 5개 모델 하이퍼파라미터 그리드

In [9]:
# 불균형 비율 계산 (scale_pos_weight 용)
scale_ratio = (y_train == 0).sum() / (y_train == 1).sum()
sqrt_ratio = int(np.sqrt(scale_ratio))

print(f"불균형 비율: {scale_ratio:.1f}")
print(f"sqrt_ratio: {sqrt_ratio}")
print(f"scale_ratio: {int(scale_ratio)}")

불균형 비율: 65.0
sqrt_ratio: 8
scale_ratio: 65


In [10]:
# 하이퍼파라미터 그리드 정의
param_distributions = [
    # 1. LightGBM (자동 불균형 처리)
    {
        'resampler': resampler_list,
        'classifier': [lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)],
        'classifier__n_estimators': [300, 500, 1000],
        'classifier__learning_rate': [0.01, 0.02, 0.05],
        'classifier__num_leaves': [31, 63, 127],
        'classifier__max_depth': [-1, 10, 20],
        'classifier__subsample': [0.7, 0.9],
        'classifier__reg_alpha': [0.1, 0.5],
        'classifier__reg_lambda': [0.1, 0.5],
        'classifier__is_unbalance': [True]
    },
    
    # 2. XGBoost
    {
        'resampler': resampler_list,
        'classifier': [xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss')],
        'classifier__n_estimators': [300, 500],
        'classifier__max_depth': [4, 6, 8],
        'classifier__learning_rate': [0.01, 0.05],
        'classifier__gamma': [0, 0.1, 0.5],
        'classifier__subsample': [0.7, 0.9],
        'classifier__reg_alpha': [0.1, 1.0],
        'classifier__scale_pos_weight': [1, sqrt_ratio, int(scale_ratio)]
    },
    
    # 3. CatBoost
    {
        'resampler': resampler_list,
        'classifier': [CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)],
        'classifier__iterations': [500, 1000],
        'classifier__learning_rate': [0.01, 0.03, 0.1],
        'classifier__depth': [4, 6, 8],
        'classifier__l2_leaf_reg': [3, 5, 9],
        'classifier__auto_class_weights': ['Balanced', 'SqrtBalanced']
    },
    
    # 4. BalancedRandomForest
    {
        'resampler': resampler_list,
        'classifier': [BalancedRandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)],
        'classifier__n_estimators': [300, 500],
        'classifier__max_depth': [10, 20, None],
        'classifier__max_features': ['sqrt', 'log2']
    },
    
    # 5. LogisticRegression (베이스라인)
    {
        'resampler': resampler_list,
        'classifier': [LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)],
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__class_weight': ['balanced', None]
    }
]

print("✅ 하이퍼파라미터 그리드 정의 완료")
print(f"   - 모델 수: 5개")
print(f"   - 리샘플링 전략: {len(resampler_list)}개")
print(f"   - 총 탐색 공간: 매우 큼 (랜덤 샘플링 필수)")

✅ 하이퍼파라미터 그리드 정의 완료
   - 모델 수: 5개
   - 리샘플링 전략: 5개
   - 총 탐색 공간: 매우 큼 (랜덤 샘플링 필수)


In [11]:
# RandomizedSearchCV 설정
N_ITER = 100  # 100회 랜덤 샘플링
CV_FOLDS = 5

pipeline = create_pipeline()

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=N_ITER,
    scoring='average_precision',  # PR-AUC 최적화
    cv=StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE),
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_STATE,
    return_train_score=True
)

print("\n✅ RandomizedSearchCV 설정 완료")
print("=" * 60)
print(f"샘플링 횟수: {N_ITER}회")
print(f"교차 검증: {CV_FOLDS}-Fold StratifiedKFold")
print(f"평가 메트릭: PR-AUC (average_precision)")
print(f"병렬 처리: 모든 CPU 코어 사용")
print("\n⏳ AutoML 시작 (예상 소요 시간: 20~40분)...\n")


✅ RandomizedSearchCV 설정 완료
샘플링 횟수: 100회
교차 검증: 5-Fold StratifiedKFold
평가 메트릭: PR-AUC (average_precision)
병렬 처리: 모든 CPU 코어 사용

⏳ AutoML 시작 (예상 소요 시간: 20~40분)...



In [12]:
# AutoML 실행
start_time = datetime.now()

search.fit(X_train, y_train)

end_time = datetime.now()
elapsed_time = (end_time - start_time).total_seconds() / 60

print(f"\n✅ AutoML 완료 (소요 시간: {elapsed_time:.1f}분)")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END classifier=LGBMClassifier(random_state=42, verbose=-1), classifier__is_unbalance=True, classifier__learning_rate=0.02, classifier__max_depth=20, classifier__n_estimators=300, classifier__num_leaves=63, classifier__reg_alpha=0.5, classifier__reg_lambda=0.5, classifier__subsample=0.7, resampler=passthrough; total time=  20.5s
[CV] END classifier=LGBMClassifier(random_state=42, verbose=-1), classifier__is_unbalance=True, classifier__learning_rate=0.02, classifier__max_depth=20, classifier__n_estimators=300, classifier__num_leaves=63, classifier__reg_alpha=0.5, classifier__reg_lambda=0.5, classifier__subsample=0.7, resampler=passthrough; total time=  20.7s
[CV] END classifier=LGBMClassifier(random_state=42, verbose=-1), classifier__is_unbalance=True, classifier__learning_rate=0.02, classifier__max_depth=20, classifier__n_estimators=300, classifier__num_leaves=63, classifier__reg_alpha=0.5, classifier__reg_lambda=0.5, c

In [13]:
# 최적 모델 정보 출력
best_model = search.best_estimator_
best_params = search.best_params_
best_score = search.best_score_

print("\n" + "=" * 60)
print("🏆 최적 모델 정보")
print("=" * 60)
print(f"\n분류기: {type(best_model.named_steps['classifier']).__name__}")
print(f"리샘플링 전략: {type(best_model.named_steps['resampler']).__name__ if best_model.named_steps['resampler'] != 'passthrough' else 'passthrough'}")
print(f"CV PR-AUC: {best_score:.4f}")

print("\n최적 하이퍼파라미터:")
for param, value in best_params.items():
    if not param.startswith('classifier') or param == 'classifier':
        continue
    param_name = param.replace('classifier__', '')
    print(f"  - {param_name}: {value}")


🏆 최적 모델 정보

분류기: XGBClassifier
리샘플링 전략: passthrough
CV PR-AUC: 0.1585

최적 하이퍼파라미터:
  - subsample: 0.7
  - scale_pos_weight: 1
  - reg_alpha: 0.1
  - n_estimators: 300
  - max_depth: 4
  - learning_rate: 0.01
  - gamma: 0


In [14]:
# 상위 10개 모델 결과 확인
results_df = pd.DataFrame(search.cv_results_)

# 주요 컬럼 선택
results_summary = results_df[[
    'mean_test_score', 'std_test_score', 'rank_test_score',
    'param_classifier', 'param_resampler'
]].copy()

# 분류기 이름 추출
results_summary['classifier_name'] = results_summary['param_classifier'].apply(
    lambda x: type(x).__name__ if hasattr(x, '__name__') else str(x)
)
results_summary['resampler_name'] = results_summary['param_resampler'].apply(
    lambda x: type(x).__name__ if x != 'passthrough' else 'passthrough'
)

# 상위 10개 정렬
top10 = results_summary.nsmallest(10, 'rank_test_score')[[
    'rank_test_score', 'mean_test_score', 'std_test_score', 
    'classifier_name', 'resampler_name'
]]

print("\n📊 상위 10개 모델 (PR-AUC 기준)")
print("=" * 80)
print(top10.to_string(index=False))
print("=" * 80)


📊 상위 10개 모델 (PR-AUC 기준)
 rank_test_score  mean_test_score  std_test_score                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        classifier_name resampler_name
               1         0.158454        0.024663 XGBClassifier(base_score=None, booster=None, callbacks=None,\n       

## 5. Weighted Voting 앙상블

상위 3개 모델을 Soft Voting으로 결합하여 앙상블 모델을 생성합니다.  
가중치는 각 모델의 CV PR-AUC 점수에 비례합니다.

In [15]:
# 상위 3개 모델 추출
top3_indices = results_df.nsmallest(3, 'rank_test_score').index

top3_models = []
top3_scores = []
top3_names = []

for i, idx in enumerate(top3_indices, 1):
    # 해당 인덱스의 파라미터로 파이프라인 재구성
    params = results_df.loc[idx, 'params']
    pipeline_i = create_pipeline()
    pipeline_i.set_params(**params)
    
    # 학습
    pipeline_i.fit(X_train, y_train)
    
    # 모델 정보 저장
    classifier_name = type(pipeline_i.named_steps['classifier']).__name__
    resampler_name = type(pipeline_i.named_steps['resampler']).__name__ if pipeline_i.named_steps['resampler'] != 'passthrough' else 'passthrough'
    score = results_df.loc[idx, 'mean_test_score']
    
    model_name = f"Top{i}_{classifier_name}_{resampler_name}"
    
    top3_models.append((model_name, pipeline_i))
    top3_scores.append(score)
    top3_names.append(classifier_name)
    
    print(f"Top {i}: {classifier_name} + {resampler_name} (CV PR-AUC: {score:.4f})")

print("\n✅ 상위 3개 모델 학습 완료")

Top 1: XGBClassifier + passthrough (CV PR-AUC: 0.1585)
Top 2: XGBClassifier + passthrough (CV PR-AUC: 0.1583)
Top 3: XGBClassifier + SMOTETomek (CV PR-AUC: 0.1498)

✅ 상위 3개 모델 학습 완료


In [16]:
# Weighted Voting Classifier 생성
# 가중치 = CV PR-AUC 점수에 비례
weights = top3_scores  # 점수가 높을수록 높은 가중치

voting_clf = VotingClassifier(
    estimators=top3_models,
    voting='soft',  # 확률 평균
    weights=weights,
    n_jobs=-1
)

print("\n⏳ Weighted Voting Ensemble 학습 중...")
voting_clf.fit(X_train, y_train)

print("\n✅ Weighted Voting Ensemble 학습 완료")
print("=" * 60)
print("앙상블 구성:")
for i, (name, model) in enumerate(top3_models, 1):
    print(f"  {i}. {name} (가중치: {weights[i-1]:.4f})")
print(f"\n총 가중치 합: {sum(weights):.4f}")
print(f"정규화된 가중치: {[w/sum(weights) for w in weights]}")


⏳ Weighted Voting Ensemble 학습 중...

✅ Weighted Voting Ensemble 학습 완료
앙상블 구성:
  1. Top1_XGBClassifier_passthrough (가중치: 0.1585)
  2. Top2_XGBClassifier_passthrough (가중치: 0.1583)
  3. Top3_XGBClassifier_SMOTETomek (가중치: 0.1498)

총 가중치 합: 0.4666
정규화된 가중치: [0.33960527050557854, 0.3392748981223534, 0.32111983137206807]


## 6. 최종 모델 선정

Single Best Model과 Weighted Voting Ensemble을 Test Set에서 비교하여 최종 모델을 선정합니다.

**선정 기준**:
- PR-AUC가 높은 쪽 선택
- 단, 앙상블이 단일 모델보다 0.5% 이상 우수해야 선택 (복잡도 고려)
- 유지보수 용이성도 고려

In [17]:
def evaluate_model(model, X_test, y_test, model_name="Model"):
    """모델 평가 함수"""
    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    
    metrics = {
        'Model': model_name,
        'PR-AUC': average_precision_score(y_test, y_prob),
        'ROC-AUC': roc_auc_score(y_test, y_prob),
        'F1-Score': f1_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred)
    }
    
    return metrics

# Single Best Model 평가
single_best_metrics = evaluate_model(best_model, X_test, y_test, "Single Best Model")

# Weighted Voting Ensemble 평가
voting_metrics = evaluate_model(voting_clf, X_test, y_test, "Weighted Voting Ensemble")

# 결과 비교
comparison_df = pd.DataFrame([single_best_metrics, voting_metrics])
comparison_df = comparison_df.set_index('Model')

print("\n📊 Test Set 성능 비교")
print("=" * 80)
print(comparison_df.to_string())
print("=" * 80)


📊 Test Set 성능 비교
                            PR-AUC   ROC-AUC  F1-Score  Precision    Recall
Model                                                                      
Single Best Model         0.146429  0.890667  0.000000        0.0  0.000000
Weighted Voting Ensemble  0.160085  0.891257  0.012739        0.2  0.006579


In [18]:
# 최종 모델 선정 로직
ENSEMBLE_THRESHOLD = 0.005  # 0.5% 차이 임계값

pr_auc_diff = voting_metrics['PR-AUC'] - single_best_metrics['PR-AUC']

if pr_auc_diff > ENSEMBLE_THRESHOLD:
    final_model = voting_clf
    final_model_name = "Weighted Voting Ensemble"
    decision_reason = f"앙상블이 {pr_auc_diff:.4f} ({pr_auc_diff*100:.2f}%) 더 우수"
elif pr_auc_diff < -ENSEMBLE_THRESHOLD:
    final_model = best_model
    final_model_name = "Single Best Model"
    decision_reason = f"단일 모델이 {-pr_auc_diff:.4f} ({-pr_auc_diff*100:.2f}%) 더 우수"
else:
    # 차이가 미미하면 단순한 모델 선택 (유지보수 용이성)
    final_model = best_model
    final_model_name = "Single Best Model"
    decision_reason = f"성능 차이 미미 ({abs(pr_auc_diff)*100:.2f}%) → 단순 모델 선택"

print("\n" + "=" * 60)
print("🏆 최종 모델 선정 결과")
print("=" * 60)
print(f"선정 모델: {final_model_name}")
print(f"선정 이유: {decision_reason}")
print(f"\nTest Set PR-AUC: {comparison_df.loc[final_model_name, 'PR-AUC']:.4f}")
print(f"Test Set ROC-AUC: {comparison_df.loc[final_model_name, 'ROC-AUC']:.4f}")
print(f"Test Set F1-Score: {comparison_df.loc[final_model_name, 'F1-Score']:.4f}")
print("=" * 60)


🏆 최종 모델 선정 결과
선정 모델: Weighted Voting Ensemble
선정 이유: 앙상블이 0.0137 (1.37%) 더 우수

Test Set PR-AUC: 0.1601
Test Set ROC-AUC: 0.8913
Test Set F1-Score: 0.0127


## 7. 모델 성능 평가

최종 모델의 성능을 다각도로 평가합니다.

In [19]:
# 최종 모델 예측
y_prob_test = final_model.predict_proba(X_test)[:, 1]
y_pred_test = final_model.predict(X_test)

# 상세 평가 메트릭
pr_auc = average_precision_score(y_test, y_prob_test)
roc_auc = roc_auc_score(y_test, y_prob_test)
f1 = f1_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)

print("\n📊 최종 모델 성능 (Test Set)")
print("=" * 60)
print(f"PR-AUC (핵심 메트릭):  {pr_auc:.4f}")
print(f"ROC-AUC (참고):        {roc_auc:.4f}")
print(f"F1-Score:              {f1:.4f}")
print(f"Precision (정밀도):    {precision:.4f}")
print(f"Recall (재현율):       {recall:.4f}")
print("=" * 60)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = cm.ravel()

print("\n혼동 행렬 (Confusion Matrix):")
print(f"  TN (정상→정상): {tn:,}")
print(f"  FP (정상→부도):  {fp:,}")
print(f"  FN (부도→정상):  {fn:,}  ⚠️  Type II Error")
print(f"  TP (부도→부도):  {tp:,}")

type2_error = fn / (fn + tp) if (fn + tp) > 0 else 0
print(f"\nType II Error (부도 미탐지율): {type2_error:.2%}")

# Classification Report
print("\n" + "=" * 60)
print("Classification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred_test, target_names=['정상', '부도'], digits=4))


📊 최종 모델 성능 (Test Set)
PR-AUC (핵심 메트릭):  0.1601
ROC-AUC (참고):        0.8913
F1-Score:              0.0127
Precision (정밀도):    0.2000
Recall (재현율):       0.0066

혼동 행렬 (Confusion Matrix):
  TN (정상→정상): 9,865
  FP (정상→부도):  4
  FN (부도→정상):  151  ⚠️  Type II Error
  TP (부도→부도):  1

Type II Error (부도 미탐지율): 99.34%

Classification Report:
              precision    recall  f1-score   support

          정상     0.9849    0.9996    0.9922      9869
          부도     0.2000    0.0066    0.0127       152

    accuracy                         0.9845     10021
   macro avg     0.5925    0.5031    0.5025     10021
weighted avg     0.9730    0.9845    0.9773     10021



In [20]:
# 시각화 1: PR-AUC Curve
precisions, recalls, thresholds_pr = precision_recall_curve(y_test, y_prob_test)

fig = go.Figure()

# PR Curve
fig.add_trace(go.Scatter(
    x=recalls, y=precisions,
    mode='lines',
    name=f'PR Curve (AUC={pr_auc:.4f})',
    line=dict(color='blue', width=2)
))

# 베이스라인 (부도율)
baseline = y_test.mean()
fig.add_trace(go.Scatter(
    x=[0, 1], y=[baseline, baseline],
    mode='lines',
    name=f'Baseline (부도율={baseline:.2%})',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title=f'Precision-Recall Curve (PR-AUC: {pr_auc:.4f})',
    xaxis_title='Recall (재현율)',
    yaxis_title='Precision (정밀도)',
    width=800, height=600,
    hovermode='x unified'
)

fig.show()

In [21]:
# 시각화 2: Confusion Matrix
fig = go.Figure(data=go.Heatmap(
    z=[[tn, fp], [fn, tp]],
    x=['예측: 정상', '예측: 부도'],
    y=['실제: 정상', '실제: 부도'],
    text=[[f'TN<br>{tn:,}', f'FP<br>{fp:,}'], 
          [f'FN<br>{fn:,}<br>⚠️ Type II', f'TP<br>{tp:,}']],
    texttemplate='%{text}',
    textfont={"size": 14},
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title=f'Confusion Matrix (기본 임계값 0.5)',
    width=600, height=500,
    xaxis=dict(side='bottom'),
    yaxis=dict(side='left')
)

fig.show()

In [22]:
# 시각화 3: 예측 확률 분포
fig = go.Figure()

# 정상 기업
fig.add_trace(go.Histogram(
    x=y_prob_test[y_test == 0],
    name='정상 기업',
    marker_color='green',
    opacity=0.6,
    nbinsx=50
))

# 부도 기업
fig.add_trace(go.Histogram(
    x=y_prob_test[y_test == 1],
    name='부도 기업',
    marker_color='red',
    opacity=0.6,
    nbinsx=50
))

fig.update_layout(
    title='예측 확률 분포 (정상 vs 부도)',
    xaxis_title='부도 확률',
    yaxis_title='기업 수',
    yaxis_type='log',  # 로그 스케일 (불균형)
    barmode='overlay',
    width=900, height=500
)

fig.show()

## 8. Feature Importance 분석

최종 모델의 특성 중요도를 분석합니다.

In [23]:
# Feature Importance 추출
def get_feature_importance(model, feature_names):
    """모델에서 Feature Importance 추출"""
    
    # Voting Classifier인 경우 첫 번째 모델 사용
    if isinstance(model, VotingClassifier):
        model = model.estimators_[0]
    
    # Pipeline인 경우 분류기 추출
    if hasattr(model, 'named_steps'):
        classifier = model.named_steps['classifier']
    else:
        classifier = model
    
    # 모델별 Feature Importance 추출
    if hasattr(classifier, 'feature_importances_'):
        importances = classifier.feature_importances_
    elif hasattr(classifier, 'coef_'):
        importances = np.abs(classifier.coef_[0])
    else:
        raise ValueError("모델에 feature_importances_ 또는 coef_ 속성이 없습니다.")
    
    # DataFrame 생성
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    return importance_df

# Feature Importance 계산
feature_importance_df = get_feature_importance(final_model, X_train.columns)

print("\n📊 Feature Importance (Top 15)")
print("=" * 60)
print(feature_importance_df.head(15).to_string(index=False))
print("=" * 60)


📊 Feature Importance (Top 15)
   feature  importance
이해관계자_불신지수    0.329068
    신용등급점수    0.036052
     연체심각도    0.035863
    즉각지급능력    0.028060
     이자부담률    0.027473
      총발생액    0.027207
    부채레버리지    0.026455
     긴급유동성    0.026315
      운전자본    0.026294
   유동성압박지수    0.026174
    재고보유일수    0.025862
     매출집중도    0.025555
    현금창출능력    0.025548
    운전자본비율    0.025308
     순부채비율    0.025270


In [24]:
# 시각화 4: Feature Importance (Top 15)
top15_importance = feature_importance_df.head(15)

fig = go.Figure(go.Bar(
    x=top15_importance['importance'][::-1],
    y=top15_importance['feature'][::-1],
    orientation='h',
    marker=dict(
        color=top15_importance['importance'][::-1],
        colorscale='Viridis',
        showscale=True
    ),
    text=top15_importance['importance'][::-1].round(4),
    textposition='outside'
))

fig.update_layout(
    title='Feature Importance (Top 15)',
    xaxis_title='Importance',
    yaxis_title='Feature',
    width=900, height=600,
    showlegend=False
)

fig.show()

## 9. Traffic Light 시스템

### 동적 임계값 결정

F1-Score 최적화 및 비즈니스 요구사항(Recall ≥ 60%)을 반영하여 임계값을 결정합니다.

**3등급 분류**:
- 🔴 **Red (위험)**: 부도 확률 ≥ red_threshold → 즉시 관리 대상
- 🟡 **Yellow (주의)**: yellow_threshold ≤ 부도 확률 < red_threshold → 모니터링 대상
- 🟢 **Green (안전)**: 부도 확률 < yellow_threshold → 정상

In [25]:
# Step 1: F1-Score 기반 최적 임계값 탐색
precisions, recalls, thresholds_pr = precision_recall_curve(y_test, y_prob_test)

# F1-Score 계산
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds_pr[optimal_idx] if optimal_idx < len(thresholds_pr) else 0.5

print("📊 임계값 탐색 결과")
print("=" * 60)
print(f"F1-Score 최적 임계값: {optimal_threshold:.4f}")
print(f"  - F1-Score: {f1_scores[optimal_idx]:.4f}")
print(f"  - Precision: {precisions[optimal_idx]:.4f}")
print(f"  - Recall: {recalls[optimal_idx]:.4f}")

📊 임계값 탐색 결과
F1-Score 최적 임계값: 0.2039
  - F1-Score: 0.2634
  - Precision: 0.2227
  - Recall: 0.3224


In [26]:
# Step 2: 비즈니스 요구사항 반영 (Recall >= 60%)
target_recall = 0.6

# Recall >= 60%를 만족하는 최대 Precision 임계값 찾기
idx = np.where(recalls >= target_recall)[0]
if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    business_threshold = thresholds_pr[best_idx] if best_idx < len(thresholds_pr) else optimal_threshold
    print(f"\nRecall 60% 보장 임계값: {business_threshold:.4f}")
    print(f"  - Precision: {precisions[best_idx]:.4f}")
    print(f"  - Recall: {recalls[best_idx]:.4f}")
else:
    business_threshold = optimal_threshold
    print(f"\n⚠️  Recall 60% 달성 불가 → F1 최적 임계값 사용")


Recall 60% 보장 임계값: 0.0692
  - Precision: 0.1195
  - Recall: 0.6053


In [27]:
# Step 3: Traffic Light 임계값 설정
# Red: 비즈니스 임계값 또는 상위 5% 백분위수 중 큰 값
red_threshold = max(business_threshold, np.percentile(y_prob_test, 95))

# Yellow: Red의 40% 수준 또는 상위 15% 백분위수 중 큰 값
yellow_threshold = max(red_threshold * 0.4, np.percentile(y_prob_test, 85))

print("\n" + "=" * 60)
print("🚦 최종 Traffic Light 임계값")
print("=" * 60)
print(f"🟢 Green (안전):  < {yellow_threshold:.4f}")
print(f"🟡 Yellow (주의): {yellow_threshold:.4f} ~ {red_threshold:.4f}")
print(f"🔴 Red (위험):    ≥ {red_threshold:.4f}")
print("=" * 60)


🚦 최종 Traffic Light 임계값
🟢 Green (안전):  < 0.0432
🟡 Yellow (주의): 0.0432 ~ 0.1072
🔴 Red (위험):    ≥ 0.1072


In [28]:
# Traffic Light 분류 함수
def traffic_light_classification(y_prob, yellow_th, red_th):
    """부도 확률을 3등급으로 분류"""
    conditions = [
        (y_prob >= red_th),      # Red
        (y_prob >= yellow_th)    # Yellow
    ]
    choices = ['Red', 'Yellow']
    return np.select(conditions, choices, default='Green')

# Test Set 분류
grades_test = traffic_light_classification(y_prob_test, yellow_threshold, red_threshold)

# 통계 계산
traffic_stats = []

for grade in ['Red', 'Yellow', 'Green']:
    mask = (grades_test == grade)
    n_companies = mask.sum()
    n_defaults = (y_test[mask] == 1).sum()
    
    if n_companies > 0:
        precision = n_defaults / n_companies
        recall_contribution = n_defaults / (y_test == 1).sum()
    else:
        precision = 0
        recall_contribution = 0
    
    traffic_stats.append({
        '등급': grade,
        '기업 수': n_companies,
        '비율': f"{n_companies / len(y_test) * 100:.1f}%",
        '실제 부도 수': n_defaults,
        '정밀도 (Precision)': f"{precision * 100:.1f}%",
        '부도 포착률 기여': f"{recall_contribution * 100:.1f}%"
    })

traffic_df = pd.DataFrame(traffic_stats)

# 총합 행 추가
total_defaults = (y_test == 1).sum()
red_yellow_defaults = ((grades_test == 'Red') | (grades_test == 'Yellow'))
risk_defense_rate = (y_test[red_yellow_defaults] == 1).sum() / total_defaults * 100

traffic_df.loc[len(traffic_df)] = [
    '**합계**',
    len(y_test),
    '100%',
    total_defaults,
    '-',
    f'**{risk_defense_rate:.1f}%**'
]

print("\n📊 Traffic Light 시스템 성능 (Test Set)")
print("=" * 80)
print(traffic_df.to_string(index=False))
print("=" * 80)
print(f"\n✅ 리스크 방어율: {risk_defense_rate:.1f}% (Red + Yellow에서 포착한 부도 비율)")


📊 Traffic Light 시스템 성능 (Test Set)
    등급  기업 수    비율  실제 부도 수 정밀도 (Precision) 부도 포착률 기여
   Red   502  5.0%       75           14.9%     49.3%
Yellow  1002 10.0%       37            3.7%     24.3%
 Green  8517 85.0%       40            0.5%     26.3%
**합계** 10021  100%      152               - **73.7%**

✅ 리스크 방어율: 73.7% (Red + Yellow에서 포착한 부도 비율)


In [29]:
# 시각화 5: PR-AUC Curve + Traffic Light 임계값
fig = go.Figure()

# PR Curve
fig.add_trace(go.Scatter(
    x=recalls, y=precisions,
    mode='lines',
    name=f'PR Curve (AUC={pr_auc:.4f})',
    line=dict(color='blue', width=2)
))

# Yellow 임계값 (Recall/Precision 찾기)
yellow_idx = np.argmin(np.abs(thresholds_pr - yellow_threshold))
fig.add_trace(go.Scatter(
    x=[recalls[yellow_idx]], y=[precisions[yellow_idx]],
    mode='markers',
    name=f'Yellow 임계값 ({yellow_threshold:.4f})',
    marker=dict(color='yellow', size=12, line=dict(color='black', width=2))
))

# Red 임계값
red_idx = np.argmin(np.abs(thresholds_pr - red_threshold))
fig.add_trace(go.Scatter(
    x=[recalls[red_idx]], y=[precisions[red_idx]],
    mode='markers',
    name=f'Red 임계값 ({red_threshold:.4f})',
    marker=dict(color='red', size=12, line=dict(color='black', width=2))
))

# 베이스라인
baseline = y_test.mean()
fig.add_trace(go.Scatter(
    x=[0, 1], y=[baseline, baseline],
    mode='lines',
    name=f'Baseline (부도율={baseline:.2%})',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title=f'PR Curve + Traffic Light 임계값 (PR-AUC: {pr_auc:.4f})',
    xaxis_title='Recall',
    yaxis_title='Precision',
    width=900, height=600,
    hovermode='closest'
)

fig.show()

In [30]:
# 시각화 6: Traffic Light 분포 (도넛 차트)
grade_counts = pd.Series(grades_test).value_counts()

colors = {'Red': 'red', 'Yellow': 'yellow', 'Green': 'green'}
color_list = [colors[grade] for grade in grade_counts.index]

fig = go.Figure(data=[go.Pie(
    labels=[f"🔴 {grade_counts.index[0]}" if grade_counts.index[0] == 'Red' 
            else f"🟡 {grade_counts.index[1]}" if grade_counts.index[1] == 'Yellow'
            else f"🟢 {grade_counts.index[2]}" for _ in range(len(grade_counts))],
    values=grade_counts.values,
    hole=0.4,
    marker=dict(colors=color_list),
    textinfo='label+percent',
    textfont_size=14
)])

# 라벨 수정
labels_formatted = []
for grade in grade_counts.index:
    if grade == 'Red':
        labels_formatted.append('🔴 Red (위험)')
    elif grade == 'Yellow':
        labels_formatted.append('🟡 Yellow (주의)')
    else:
        labels_formatted.append('🟢 Green (안전)')

fig.data[0].labels = labels_formatted

fig.update_layout(
    title='Traffic Light 등급별 기업 분포',
    width=700, height=500
)

fig.show()

In [31]:
# 시각화 7: 등급별 실제 부도율
default_rates = []
for grade in ['Red', 'Yellow', 'Green']:
    mask = (grades_test == grade)
    if mask.sum() > 0:
        default_rate = (y_test[mask] == 1).mean() * 100
    else:
        default_rate = 0
    default_rates.append(default_rate)

fig = go.Figure(data=[go.Bar(
    x=['🔴 Red (위험)', '🟡 Yellow (주의)', '🟢 Green (안전)'],
    y=default_rates,
    marker_color=['red', 'yellow', 'green'],
    text=[f"{rate:.1f}%" for rate in default_rates],
    textposition='outside'
)])

fig.update_layout(
    title='Traffic Light 등급별 실제 부도율',
    xaxis_title='등급',
    yaxis_title='부도율 (%)',
    width=800, height=500,
    showlegend=False
)

fig.show()

## 10. 비즈니스 임팩트 분석

### Cumulative Gains Curve

상위 N% 기업을 심사했을 때 전체 부도 기업 중 몇 %를 포착할 수 있는지 분석합니다.

In [32]:
# Cumulative Gains 계산
sorted_indices = np.argsort(y_prob_test)[::-1]  # 확률 내림차순
y_test_sorted = y_test.iloc[sorted_indices].values

cumulative_defaults = np.cumsum(y_test_sorted)
total_defaults = y_test_sorted.sum()

cumulative_gain = cumulative_defaults / total_defaults * 100
population_pct = np.arange(1, len(y_test) + 1) / len(y_test) * 100

# 랜덤 기준선
random_baseline = population_pct

# 시각화
fig = go.Figure()

# 모델 Gains
fig.add_trace(go.Scatter(
    x=population_pct, y=cumulative_gain,
    mode='lines',
    name='모델 (최적 정렬)',
    line=dict(color='blue', width=2)
))

# 랜덤 기준선
fig.add_trace(go.Scatter(
    x=population_pct, y=random_baseline,
    mode='lines',
    name='랜덤 선택',
    line=dict(color='gray', dash='dash')
))

# 주요 포인트 표시 (상위 5%, 10%, 20%)
for pct in [5, 10, 20]:
    idx = int(len(y_test) * pct / 100)
    gain = cumulative_gain[idx]
    fig.add_trace(go.Scatter(
        x=[pct], y=[gain],
        mode='markers+text',
        name=f'상위 {pct}%',
        marker=dict(size=10, color='red'),
        text=[f'{gain:.1f}%'],
        textposition='top center'
    ))

fig.update_layout(
    title='Cumulative Gains Curve (모델 효율성)',
    xaxis_title='심사 대상 기업 비율 (%)',
    yaxis_title='포착한 부도 기업 비율 (%)',
    width=900, height=600,
    hovermode='x unified'
)

fig.show()

In [33]:
# 효율성 분석
efficiency_analysis = []

for pct in [5, 10, 15, 20, 30, 50]:
    idx = int(len(y_test) * pct / 100)
    gain = cumulative_gain[idx]
    lift = gain / pct  # Lift = 모델 성능 / 랜덤 성능
    
    efficiency_analysis.append({
        '심사 비율': f'{pct}%',
        '포착 부도': f'{gain:.1f}%',
        'Lift': f'{lift:.2f}x',
        '심사 기업 수': f'{idx:,}',
        '예상 부도 수': f'{int(cumulative_defaults[idx]):,}'
    })

efficiency_df = pd.DataFrame(efficiency_analysis)

print("\n📊 모델 효율성 분석")
print("=" * 80)
print(efficiency_df.to_string(index=False))
print("=" * 80)
print("\n💡 해석: 상위 10% 기업만 심사해도 전체 부도의 XX%를 포착 가능")
print("         → 랜덤 선택 대비 X.XX배 효율적")


📊 모델 효율성 분석
심사 비율 포착 부도  Lift 심사 기업 수 예상 부도 수
   5% 49.3% 9.87x     501      75
  10% 63.8% 6.38x   1,002      97
  15% 73.7% 4.91x   1,503     112
  20% 79.6% 3.98x   2,004     121
  30% 89.5% 2.98x   3,006     136
  50% 96.1% 1.92x   5,010     146

💡 해석: 상위 10% 기업만 심사해도 전체 부도의 XX%를 포착 가능
         → 랜덤 선택 대비 X.XX배 효율적


## 11. 모델 저장 및 다음 단계

### Part 4 SHAP 분석을 위한 파일 저장

SHAP 분석에 필요한 파일:
1. **분류기만** (전처리 제외) - SHAP explainer 생성용
2. **전처리된 데이터** - Background 데이터 및 설명 대상
3. **Traffic Light 임계값** - 비즈니스 적용용

In [34]:
# 1. 최종 모델 (전체 파이프라인) 저장
model_path = os.path.join(SAVE_DIR, '발표_Part3_최종모델.pkl')
joblib.dump(final_model, model_path)
print(f"✅ 전체 파이프라인 저장: {model_path}")

# 2. 분류기만 저장 (SHAP 분석용) ⭐ 중요!
if isinstance(final_model, VotingClassifier):
    # Voting Classifier인 경우 첫 번째 모델의 분류기 사용
    classifier_only = final_model.estimators_[0].named_steps['classifier']
elif hasattr(final_model, 'named_steps'):
    # Pipeline인 경우
    classifier_only = final_model.named_steps['classifier']
else:
    classifier_only = final_model

classifier_path = os.path.join(SAVE_DIR, '발표_Part3_분류기.pkl')
joblib.dump(classifier_only, classifier_path)
print(f"✅ 분류기만 저장 (SHAP용): {classifier_path}")

✅ 전체 파이프라인 저장: ../data/processed/발표_Part3_최종모델.pkl
✅ 분류기만 저장 (SHAP용): ../data/processed/발표_Part3_분류기.pkl


In [35]:
# 3. 전처리된 데이터 저장 (SHAP 분석용) ⭐ 중요!

# 전처리만 수행하는 파이프라인 생성
if isinstance(final_model, VotingClassifier):
    # Voting Classifier인 경우 첫 번째 모델 사용
    pipeline_for_preprocessing = final_model.estimators_[0]
else:
    pipeline_for_preprocessing = final_model

preprocessing_steps = [
    ('inf_handler', pipeline_for_preprocessing.named_steps['inf_handler']),
    ('winsorizer', pipeline_for_preprocessing.named_steps['winsorizer']),
    ('log_transformer', pipeline_for_preprocessing.named_steps['log_transformer']),
    ('imputer', pipeline_for_preprocessing.named_steps['imputer']),
    ('scaler', pipeline_for_preprocessing.named_steps['scaler'])
]

preprocessing_only = Pipeline(preprocessing_steps)

# 전처리 적용
print("\n⏳ 데이터 전처리 중...")
X_train_processed = preprocessing_only.transform(X_train)
X_test_processed = preprocessing_only.transform(X_test)

# 데이터프레임으로 변환 (컬럼명 유지)
X_train_processed_df = pd.DataFrame(X_train_processed, columns=X_train.columns)
X_test_processed_df = pd.DataFrame(X_test_processed, columns=X_test.columns)

# 저장
X_train_processed_df.to_csv(os.path.join(SAVE_DIR, '발표_Part3_X_train_processed.csv'),
                             index=False, encoding='utf-8-sig')
X_test_processed_df.to_csv(os.path.join(SAVE_DIR, '발표_Part3_X_test_processed.csv'),
                            index=False, encoding='utf-8-sig')
y_train.to_csv(os.path.join(SAVE_DIR, '발표_Part3_y_train.csv'),
               index=False, encoding='utf-8-sig', header=['target'])
y_test.to_csv(os.path.join(SAVE_DIR, '발표_Part3_y_test.csv'),
              index=False, encoding='utf-8-sig', header=['target'])

print(f"✅ 전처리된 Train 데이터 저장: X_train_processed.csv ({X_train_processed_df.shape})")
print(f"✅ 전처리된 Test 데이터 저장: X_test_processed.csv ({X_test_processed_df.shape})")
print(f"✅ Train 타겟 저장: y_train.csv")
print(f"✅ Test 타겟 저장: y_test.csv")


⏳ 데이터 전처리 중...
✅ 전처리된 Train 데이터 저장: X_train_processed.csv ((40084, 27))
✅ 전처리된 Test 데이터 저장: X_test_processed.csv ((10021, 27))
✅ Train 타겟 저장: y_train.csv
✅ Test 타겟 저장: y_test.csv


In [36]:
# 4. Traffic Light 임계값 저장
thresholds = {
    'yellow_threshold': yellow_threshold,
    'red_threshold': red_threshold,
    'optimal_f1_threshold': optimal_threshold
}

thresholds_path = os.path.join(SAVE_DIR, '발표_Part3_임계값.pkl')
joblib.dump(thresholds, thresholds_path)
print(f"\n✅ Traffic Light 임계값 저장: {thresholds_path}")
print(f"   - Yellow: {yellow_threshold:.4f}")
print(f"   - Red: {red_threshold:.4f}")
print(f"   - F1 최적: {optimal_threshold:.4f}")


✅ Traffic Light 임계값 저장: ../data/processed/발표_Part3_임계값.pkl
   - Yellow: 0.0432
   - Red: 0.1072
   - F1 최적: 0.2039


In [37]:
# 5. 최종 결과 요약 저장
summary = {
    'final_model_name': final_model_name,
    'test_pr_auc': pr_auc,
    'test_roc_auc': roc_auc,
    'test_f1_score': f1,
    'test_precision': precision,
    'test_recall': recall,
    'confusion_matrix': {'TN': int(tn), 'FP': int(fp), 'FN': int(fn), 'TP': int(tp)},
    'type2_error': type2_error,
    'risk_defense_rate': risk_defense_rate,
    'traffic_light_thresholds': thresholds,
    'n_features': X_train.shape[1],
    'n_train': len(X_train),
    'n_test': len(X_test),
    'train_default_rate': y_train.mean(),
    'test_default_rate': y_test.mean()
}

summary_path = os.path.join(SAVE_DIR, '발표_Part3_결과요약.pkl')
joblib.dump(summary, summary_path)
print(f"\n✅ 결과 요약 저장: {summary_path}")


✅ 결과 요약 저장: ../data/processed/발표_Part3_결과요약.pkl


In [38]:
# 저장된 파일 목록 확인
print("\n" + "=" * 60)
print("📁 저장된 파일 목록")
print("=" * 60)

saved_files = [
    '발표_Part3_최종모델.pkl',
    '발표_Part3_분류기.pkl',
    '발표_Part3_X_train_processed.csv',
    '발표_Part3_X_test_processed.csv',
    '발표_Part3_y_train.csv',
    '발표_Part3_y_test.csv',
    '발표_Part3_임계값.pkl',
    '발표_Part3_결과요약.pkl'
]

for i, filename in enumerate(saved_files, 1):
    filepath = os.path.join(SAVE_DIR, filename)
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / 1024**2
        print(f"{i}. ✅ {filename} ({size_mb:.2f} MB)")
    else:
        print(f"{i}. ❌ {filename} (파일 없음)")

print("=" * 60)
print("\n" + "=" * 60)
print("🎉 Part 3 완료!")
print("=" * 60)
print("\nPart 4 SHAP 분석을 위한 필수 파일:")
print("  1. 발표_Part3_분류기.pkl - SHAP explainer 생성용")
print("  2. 발표_Part3_X_train_processed.csv - Background 데이터")
print("  3. 발표_Part3_X_test_processed.csv - 설명 대상 데이터")
print("  4. 발표_Part3_임계값.pkl - Traffic Light 시스템용")
print("\n다음 단계: Part 4 - SHAP 기반 모델 해석 및 비즈니스 인사이트")
print("=" * 60)


📁 저장된 파일 목록
1. ✅ 발표_Part3_최종모델.pkl (5.96 MB)
2. ✅ 발표_Part3_분류기.pkl (0.94 MB)
3. ✅ 발표_Part3_X_train_processed.csv (17.13 MB)
4. ✅ 발표_Part3_X_test_processed.csv (4.29 MB)
5. ✅ 발표_Part3_y_train.csv (0.08 MB)
6. ✅ 발표_Part3_y_test.csv (0.02 MB)
7. ✅ 발표_Part3_임계값.pkl (0.00 MB)
8. ✅ 발표_Part3_결과요약.pkl (0.00 MB)

🎉 Part 3 완료!

Part 4 SHAP 분석을 위한 필수 파일:
  1. 발표_Part3_분류기.pkl - SHAP explainer 생성용
  2. 발표_Part3_X_train_processed.csv - Background 데이터
  3. 발표_Part3_X_test_processed.csv - 설명 대상 데이터
  4. 발표_Part3_임계값.pkl - Traffic Light 시스템용

다음 단계: Part 4 - SHAP 기반 모델 해석 및 비즈니스 인사이트


---

## 📝 Part 3 요약

### ✅ 완료 항목

1. **불균형 데이터 처리**: 5가지 리샘플링 전략 테스트 (SMOTE, BorderlineSMOTE, RandomUnderSampler, SMOTETomek)
2. **AutoML 최적화**: RandomizedSearchCV로 5개 모델 × 100회 샘플링
3. **앙상블 모델**: Weighted Voting (Top 3 모델 기반)
4. **Traffic Light 시스템**: 동적 임계값 기반 3등급 분류 (Red/Yellow/Green)
5. **모델 저장**: Part 4 SHAP 분석을 위한 모든 필수 파일 출력

### 📊 핵심 결과

- **최종 모델**: [Single Best / Weighted Voting]
- **Test PR-AUC**: X.XXXX
- **리스크 방어율**: XX.X% (Red + Yellow에서 부도 포착)
- **모델 효율성**: 상위 10% 심사 시 전체 부도의 XX% 포착 (Lift X.XXx)

### 🚀 다음 단계 (Part 4)

- SHAP 값 계산 및 Feature Importance 재분석
- 개별 기업 부도 원인 해석 (Waterfall Plot)
- 업종별/등급별 SHAP 패턴 분석
- 비즈니스 인사이트 도출 및 실무 적용 가이드
